In [1]:
import pandas as pd
from dotenv import dotenv_values
from pathlib import Path
from datasets import Dataset

config = dotenv_values("./../../config/.env") # take environment variables from .env.
base_path = Path(config["BASE_PATH"])
data_path = base_path/"data"

In [4]:
df = pd.read_pickle(data_path/"preprocessed"/"paragraphs_clean.pkl")
df.report_id = df.report_id.astype(str)
dataset = Dataset.from_pandas(df, preserve_index=False)
dataset = dataset.train_test_split(test_size=100_000, seed=950808)
dataset["zero-shot"] = dataset.pop("test")
train_split = dataset.pop("train")
train_split = train_split.train_test_split(test_size=100_000, seed=950808)
dataset["active-learning"] = train_split.pop("test")
dataset["evaluate"] = train_split.pop("train")
dataset.save_to_disk(data_path/"preprocessed"/"dataset")